In [8]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split


In [2]:
path = '../data/'

dataTrain = pd.read_csv(path+'dataTrain.csv')
dataNoLabel = pd.read_csv(path+'dataNoLabel.csv')
dataA = pd.read_csv(path+'dataA.csv')
submit_example_A = pd.read_csv(path+'submit_example_A.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
feature_cols = dataTrain.columns[1:-1]
lbls = {}
for f in feature_cols:
    lbl = LabelEncoder()
    lbl.fit(sorted(dataTrain[f].tolist() + dataNoLabel[f].tolist() + dataA[f].tolist()))
    lbls[f] = lbl

In [4]:
features_to_use =  feature_cols

X = dataTrain[features_to_use]
y = dataTrain['label']
X_pred = dataA.copy()

In [5]:
#feature mapping
for f in features_to_use:
    if f in ['f3']:
        X['f3'] = X['f3'].apply(lambda x:{'low':0,'mid':1,'high':2}[x])
        X_pred['f3'] = X_pred['f3'].apply(lambda x:{'low':0,'mid':1,'high':2}[x])
    else:
        X[f] = lbls[f].transform(X[f])
        X_pred[f] = lbls[f].transform(X_pred[f])

/data/services/anaconda3/envs/franky_tfgpu/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/data/services/anaconda3/envs/franky_tfgpu/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
#split train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train&test
params = {
            'boosting': 'gbdt',
            'num_leaves': 15, 
            'num_iterations': 100,
            'learning_rate': 0.1,
            'objective': 'binary',
            'metrics':'auc',
            'seed': 1,
            'verbose': 0,}

lgb_train = lgb.Dataset(X_train, label=y_train)

clf = lgb.train(params, lgb_train, num_round,verbose_eval=1)
y_pred_lgb = clf.predict(X_test)

/data/services/anaconda3/envs/franky_tfgpu/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/data/services/anaconda3/envs/franky_tfgpu/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.194215 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [ ]:
#predict
y_out_lgb = clf.predict(X_pred[features_to_use])

submit_example_A['label'] = y_out_lgb
submit_example_A.to_csv('baseline_submission.csv',sep=',',index=False)